# Simulation Modelling

In [70]:
%run LogFileProcessing.ipynb

## Conversion Helpers

In [71]:
# Convert a string like 'Mbps' to its numerical value in bytes (e.g. Kbps -> 1000bps)
def getMultiplierFromPrefix(s):
    titleCaseString = s[0].upper() + s[1:].lower()
    assert s in ['Bps', 'Kbps', 'Mbps', 'Gbps']
    
    if titleCaseString == 'Bps':
        return 1
    elif titleCaseString == 'Kbps':
        return 1000
    elif titleCaseString == 'Mbps':
        return 1000000
    else:
        assert titleCaseString == 'Gbps' 
        return 1000000000


# Convert a bitrate string like 1Mbps to a bps measure
def bpsRateFromString(rateString):
    numericPart = rateString.strip(string.ascii_letters)
    unitPart = rateString.strip(string.digits)
    
    assert rateString == numericPart + unitPart
    
    multiplier = getMultiplierFromPrefix(unitPart)
    return int(numericPart) * multiplier


# Convert a value in bps to a print-able string
def bpsRateToString(rateNum):
    val = 0
    postfix = ''
    if rateNum < 1000:
        val = int(rateNum)
        postfix = 'bps'
    elif rateNum < 1000000:
        val = int(rateNum / 1000)
        postfix = 'Kbps'
    else:
        val = int(rateNum / 1000000)
        postfix = 'Mbps'
    return str(val) + postfix

In [72]:
print (bpsRateFromString('999Bps'))
print (bpsRateFromString('2Kbps'))
print (bpsRateFromString('1Mbps'))

999
2000
1000000


## Simulation Class Definition and Helpers

In [13]:
def isSimulationDirectory(parentDir):
    # Assume that if one log file exists, so do the others
    return "cl0_adaptationLog.txt" in os.listdir(parentDir)

In [81]:
class Simulation:
    
    # Create a simulation object to represent the given directory
    def __init__(self, dirPath):
        self.path = dirPath
        
        relativePath = getRelativePath(dirPath)
        params = relativePath.split('\\')

        self.proto = params[0]
        self.algo = params[1]
        
        self.rateStr = params[2]
        self.rateVal = bpsRateFromString(self.rateStr)

        if (hasError(dirPath)) :
            self.errRate = float(params[3])
            self.pacing = params[4] == 'pacing'
        else:
            self.errRate = 0
            self.pacing = params[3] == 'pacing'
    
    
    def getErrorPercent(self):
        return self.errRate * 100
    
    
    # Return the number of successfully-downloaded segments
    def countSegments(self):
        _, rows = readDownloadLog(self.path)
        return len(rows)
    
    
    def duration(self):
        return self.countSegments() * 2
    
    
    # Implement the less-than operator so we can sort
    def __lt__(self, other):
        if self.proto < other.proto:
            return True
        elif other.proto < self.proto:
            return False

        if self.algo < other.algo:
            return True
        elif other.algo < self.algo:
            return False

        if self.rateVal != other.rateVal:
            return self.rateVal < other.rateVal

        if self.errRate != other.errRate:
            return self.errRate < other.errRate

        if self.pacing == other.pacing:
            return False # Equal
        elif self.pacing:
            return True
        else:
            return False

In [83]:
# Print out all of the simulation's data (for testing)
def printSimulation(simulation):
#     print (simulation.path)
    errStr = str(sim.getErrorPercent()) + '% loss' 
    pacingStr = 'pacing' if sim.pacing else "no pacing"
    result = "{} | {} | {} ({}bps) | {} | {}".format(sim.proto, sim.algo, sim.rateStr, sim.rateVal, errStr, pacingStr)
    print (result)


print ("QUIC with loss sample")
sim = Simulation(sampleTcpLossDir)
print (sim.path)
printSimulation (sim)

QUIC with loss sample
C:\Users\Jamie\Documents\UVic\4th Year\Fall 2021\CSC 499 - QUIC\jupyter-csc499\499-Visualization\data\benchmarking-loss\TCP\festive\2Mbps\0.05\no-pacing\
TCP | festive | 2Mbps (2000000bps) | 5.0% loss | no pacing


In [88]:
print(sim.duration())
print(sim.countSegments())

444
222


## Sample Simulations for Testing

In [89]:
sampleQuicSim = Simulation(sampleQuicDir)
sampleSlowQuicSim = Simulation(sampleQuicDirSlow)
sampleLossyQuicSim = Simulation(sampleQuicLossDir)

sampleTcpSim = Simulation(sampleTcpDir)
sampleSlowTcpSim = Simulation(sampleTcpDirSlow)
sampleLossyTcpSim = Simulation(sampleTcpLossDir)

## Enumerating Simulations

In [75]:
# Some helpful predicates for enumerating certain types of simulations
def isQuicSimulation(sim):
    return sim.proto == 'QUIC'


def isTcpSimulation(sim):
    return not isQuicSimulation(sim)


def quicSimulationHasNSegments(sim, n):
    return isQuicSimulation(sim) and sim.countSegments() >= n

In [76]:
# Enumerate all simulations in any subdirectory of topDir 
# which satisfy the given predicate
def findSimulations(topDir, pred):
    simulations = []
    for path, subdirs, files in os.walk(topDir):
        path = path + '\\'
        if isSimulationDirectory(path):
            sim = Simulation(path)
            if pred(sim):
                simulations.append(sim)
    
    return sorted(simulations)


# Enumerate all simulations in any subdirectory of topDir which ran until
# at least minSegmentCount segments were downloaded by the client.
def findSimulationsWithMinSegCount(topDir, minSegmentCount):
    pred = lambda sim : quicSimulationHasNSegments(sim, minSegmentCount)
    return findSimulations(topDir, pred)

In [77]:
sims = findSimulationsWithMinSegCount(quicLossDir, 150)
for sim in sims:
    printSimulation(sim)

QUIC | festive | 1Mbps (1000000bps) | 0.0% loss | pacing
QUIC | festive | 1Mbps (1000000bps) | 1.0% loss | no pacing
QUIC | festive | 2Mbps (2000000bps) | 5.0% loss | no pacing
QUIC | tobasco | 500Kbps (500000bps) | 1.0% loss | pacing
QUIC | tobasco | 2Mbps (2000000bps) | 2.0% loss | pacing
QUIC | tobasco | 3Mbps (3000000bps) | 2.0% loss | no pacing
QUIC | tobasco | 3Mbps (3000000bps) | 5.0% loss | pacing
QUIC | tobasco | 5Mbps (5000000bps) | 5.0% loss | no pacing


In [80]:
def getTcpSimulationAnalog(quicSim):
    tcpDir = findTcpAnalogPath(quicSim.path)
    return Simulation(tcpDir)


def getTcpSimulations(quicSimulations):
    return [getTcpSimulationAnalog(sim) for sim in quicSimulations]


# Enumerate (quic, tcp) pairs of simulations for which the given
# predicate is satisfied
def findSimulationPairs(topDir, pred):
    # Enumerate all QUIC simulations that satisfy pred first,
    # then find all the corresponding TCP ones.
    quicOnlyPred = lambda sim : pred(sim) and isQuicSimulation(sim)
    quicSimulations = findSimulations(topDir, pred)

    tcpSimulations = getTcpSimulations(quicSimulations)

    return zip(quicSimulations, tcpSimulations)

## Simulation String Formatting

In [73]:
# Logic to format a file path into a nicer title string
def getDescription(sim):
    pacingStr = 'with pacing' if sim.pacing else "no pacing"
    return "{} over {} @ {} with {}% loss ({})".format(sim.algo, sim.proto, sim.rateStr, sim.getErrorPercent(), pacingStr)

In [91]:
print (getDescription(sampleQuicSim))
print (getDescription(sampleLossyQuicSim))

festive over QUIC @ 2Mbps with 0% loss (with pacing)
festive over QUIC @ 2Mbps with 5.0% loss (no pacing)
